In [16]:
import pandas as pd
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from gensim.parsing.preprocessing import remove_stopwords
import string
import traceback
from scipy.spatial.distance import cdist
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import pipeline
import numpy as np



In [2]:
import nltk

# Uncomment the line below and run it once to download the necessary data
nltk.download('punkt')

[nltk_data] Error loading punkt: <urlopen error [WinError 10054] An
[nltk_data]     existing connection was forcibly closed by the remote
[nltk_data]     host>


False

In [3]:

# Load CSV file into a DataFrame
df = pd.read_csv("bigBasketProducts.csv")
df

,index,product,category,sub_category,brand,sale_price,market_price,type,rating,description
0,1,Garlic Oil - Vegetarian Capsule 500 mg,Beauty & Hygiene,Hair Care,Sri Sri Ayurveda,220.00,220.0,Hair Oil & Serum,4.1,This Product contains Garlic Oil that is known...
1,2,Water Bottle - Orange,"Kitchen, Garden & Pets",Storage & Accessories,Mastercook,180.00,180.0,Water & Fridge Bottles,2.3,"Each product is microwave safe (without lid), ..."
2,3,"Brass Angle Deep - Plain, No.2",Cleaning & Household,Pooja Needs,Trm,119.00,250.0,Lamp & Lamp Oil,3.4,"A perfect gift for all occasions, be it your m..."
3,4,Cereal Flip Lid Container/Storage Jar - Assort...,Cleaning & Household,Bins & Bathroom Ware,Nakoda,149.00,176.0,"Laundry, Storage Baskets",3.7,Multipurpose container with an attractive desi...
4,5,Creme Soft Soap - For Hands & Body,Beauty & Hygiene,Bath & Hand Wash,Nivea,162.00,162.0,Bathing Bars & Soaps,4.4,Nivea Creme Soft Soap gives your skin the best...
...,...,...,...,...,...,...,...,...,...,...
27550,27551,"Wottagirl! Perfume Spray - Heaven, Classic",Beauty & Hygiene,Fragrances & Deos,Layerr,199.20,249.0,Perfume,3.9,Layerr brings you Wottagirl Classic fragrant b...
27551,27552,Rosemary,Gourmet & World Food,Cooking & Baking Needs,Puramate,67.50,75.0,"Herbs, Seasonings & Rubs",4.0,Puramate rosemary is enough to transform a dis...
27552,27553,Peri-Peri Sweet Potato Chips,Gourmet & World Food,"Snacks, Dry Fruits, Nuts",FabBox,200.00,200.0,Nachos & Chips,3.8,We have taken the richness of Sweet Potatoes (...
27553,27554,Green Tea - Pure Original,Beverages,Tea,Tetley,396.00,495.0,Tea Bags,4.2,"Tetley Green Tea with its refreshing pure, ori..."


In [4]:
# Handling NaN values in the text data
df['product'] = df['product'].fillna('')
df['category'] = df['category'].fillna('')
df['sub_category'] = df['sub_category'].fillna('')
df['brand'] = df['brand'].fillna('')
df['description'] = df['description'].fillna('')

In [5]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Error loading punkt: <urlopen error [WinError 10054] An
[nltk_data]     existing connection was forcibly closed by the remote
[nltk_data]     host>
[nltk_data] Error loading stopwords: <urlopen error [WinError 10054]
[nltk_data]     An existing connection was forcibly closed by the
[nltk_data]     remote host>


False

In [6]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
# Function to remove stopwords and tokenize text
def remove_stopwords(text):
    stop_words = set(stopwords.words("english"))
    return [word for word in text if word.lower() not in stop_words]

# Tokenize and preprocess the product descriptions along with other relevant fields
tokenized_descriptions = [
    word_tokenize(str(description).lower()) +
    word_tokenize(str(category).lower()) +
    word_tokenize(str(sub_category).lower()) +
    word_tokenize(str(brand).lower())
    for description, category, sub_category, brand in zip(
        df["description"],
        df["category"],
        df["sub_category"],
        df["brand"],
    )
]


In [7]:
# Remove punctuation from tokens
translator = str.maketrans("", "", string.punctuation)
tokenized_descriptions = [
    [token.translate(translator) for token in tokens]
    for tokens in tokenized_descriptions
]

# Flatten the list of tokens
tokenized_descriptions = [token for tokens in tokenized_descriptions for token in tokens]

# Train Word2Vec model
word2vec_model = Word2Vec(
    sentences=[tokenized_descriptions], vector_size=300, window=1, min_count=2
)

In [8]:
from nltk.tokenize import word_tokenize
import string
sample_description = "This Product contains Garlic Oil that is known to help proper digestion, maintain proper cholesterol levels, support cardiovascular and also build immunity."

concatenated_text = (
    sample_description.lower()
    + " "
    + " ".join(["category", "sub_category", "brand"])
)

# Tokenize and preprocess the concatenated text
cleaned_sample_tokens = [
    token.translate(translator)
    for token in word_tokenize(concatenated_text)
    if token.isalnum()  # Filter out non-alphanumeric tokens
]

# Get vector for the description
vector_for_description = word2vec_model.wv[cleaned_sample_tokens]

# # Save the Word2Vec model for future use
# word2vec_model.save("word2vec_6.model")

In [9]:
# Save vectors to CSV file
vectors_df = pd.DataFrame(word2vec_model.wv.vectors, columns=[f"dim_{i}" for i in range(100)])
# vectors_df.to_csv("vectors_6.csv", index=False)
vectors_df

,dim_0,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,...,dim_90,dim_91,dim_92,dim_93,dim_94,dim_95,dim_96,dim_97,dim_98,dim_99
0,-0.045967,0.069869,0.021825,0.082724,0.005852,-0.117093,0.156167,0.195583,-0.117335,-0.117953,...,0.132317,0.086057,0.049858,0.034563,0.197261,0.090080,0.017357,-0.078351,0.001726,0.028014
1,-0.034343,0.042121,0.013910,0.047260,0.016265,-0.067516,0.083265,0.110158,-0.065032,-0.069927,...,0.073542,0.045687,0.028781,0.011123,0.101206,0.049671,0.020515,-0.041438,-0.009926,0.016055
2,-0.021900,0.034292,0.000623,0.033043,0.014707,-0.042178,0.064485,0.088297,-0.060478,-0.046131,...,0.055742,0.045522,0.031159,0.012259,0.094995,0.044873,0.018069,-0.030921,0.007186,0.003158
3,-0.026350,0.036121,0.006317,0.025837,0.010876,-0.046806,0.059254,0.078978,-0.036374,-0.051666,...,0.041623,0.031153,0.017015,0.010666,0.081313,0.031979,0.016618,-0.034009,-0.002900,0.002356
4,-0.024169,0.025969,-0.000939,0.025203,0.009226,-0.033895,0.055113,0.069595,-0.045637,-0.034126,...,0.050516,0.026786,0.022189,0.009106,0.070578,0.022655,0.016092,-0.020255,0.000369,0.010491
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24116,-0.001663,-0.000679,0.004910,0.002867,-0.005917,-0.008226,0.009866,-0.003445,-0.005183,-0.003245,...,0.008163,-0.000405,0.002196,-0.005369,-0.008971,-0.008198,0.004414,0.003118,-0.004907,-0.006214
24117,-0.002094,-0.006869,-0.005107,0.007017,0.005580,-0.005876,-0.008876,-0.003369,-0.001968,0.009257,...,0.004947,0.007990,-0.009439,0.007582,0.009644,-0.004721,-0.008918,-0.006134,-0.005178,0.000734
24118,0.009581,0.005538,-0.002029,-0.009818,0.006070,0.002822,0.002714,0.008273,0.004678,0.002147,...,0.004866,0.001431,0.009250,-0.001279,-0.003418,0.006065,-0.009151,0.000771,-0.004689,0.004257
24119,-0.000750,0.004496,0.002613,-0.009316,-0.009537,0.003055,-0.005434,-0.002579,0.001850,-0.001467,...,0.007837,-0.008015,-0.007610,0.004978,-0.003510,0.001162,0.002109,-0.004978,-0.002213,0.007406


In [10]:
# import subprocess

# # Replace 'package_name' with the name of the package you want to install
# package_name = 'transformers'

# try:
#     # Execute pip install command
#     subprocess.check_call(["pip", "install", package_name])
#     print(f"Successfully installed {package_name}")
# except subprocess.CalledProcessError:
#     print(f"Error installing {package_name}")

In [11]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load pre-trained GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")



In [19]:
from flask import Flask, request, jsonify, render_template

app = Flask(__name__)

@app.route("/get_ans", methods=["POST", "GET"])
def get_answer():
    try:
        if request.method == "POST":
            data = request.get_json()

            # Check if the 'query' key exists in the received JSON data
            if 'query' in data:
                user_query = data["query"]
                cleaned_query_tokens = [
                    token.translate(str.maketrans("", "", string.punctuation))
                    for token in word_tokenize(user_query.lower())
                    if token.isalnum()
                ]
                user_query_vector = np.mean(
                    [word2vec_model.wv[token] for token in cleaned_query_tokens if token in word2vec_model.wv],
                    axis=0
                )
                 # Find the most similar vector using cosine similarity
                similarities = 1 - cdist([user_query_vector], vectors_df.values, 'cosine')
                most_similar_index = np.argmax(similarities)
                # Modify this part based on your actual processing logic
                most_similar_description = df.loc[most_similar_index, 'description']
                
                response_message = f"Received user query: {most_similar_description}"

                return jsonify({"message": response_message})
            else:
                return jsonify({"error": "Missing 'query' key in the JSON data."})

    except Exception as e:
        return jsonify({"error": f"An error occurred: {str(e)}"})

@app.route("/", methods=["GET"])
def home():
    return render_template("home.html")

if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Nov/2023 16:30:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2023 16:30:39] "POST /get_ans HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2023 16:36:38] "POST /get_ans HTTP/1.1" 200 -
